### Scarichiamo le principali librerie che ci serviranno:

In [37]:
%pip install selenium
%pip install webdriver-manager
%pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Importiamo le librerie da usare:

In [38]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import locale
import datetime
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from lxml import etree

### Inizializziamo l'ambiente per lo scraping:

In [39]:
# Imposta in italiano la formattazione della data
locale.setlocale(locale.LC_TIME, 'it_IT.UTF-8')

# Percorso del Desktop dell'utente
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')

# Percorso della cartella Dataset sul Desktop
dataset_folder = os.path.join(desktop_path, 'Dataset')

# Percorso del file NormattivaLinks
normattiva_links_file = os.path.join(os.path.expanduser('~'), 'Desktop', 'NormattivaLinks.txt')

# Controlla se il file esiste, se no lo crea
if not os.path.isfile(normattiva_links_file):
    open(normattiva_links_file, 'w').close()

# Verifica se la cartella "Dataset" esiste
if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)
    print(f"Cartella 'Dataset' creata in {dataset_folder}")
else:
    print(f"Cartella 'Dataset' già presente in {dataset_folder}")

Cartella 'Dataset' già presente in C:\Users\burre\Desktop\Dataset


In [40]:
# Impostazioni del browser Firefox
options = Options()
options.add_argument("--headless")  # Forza la modalità headless (no GUI)
service = Service()

# Configura le preferenze per il download automatico in Firefox
profile = webdriver.FirefoxProfile()  # Crea un profilo per Firefox
profile.set_preference("browser.download.folderList", 2)  # Imposta la cartella personalizzata
profile.set_preference("browser.download.manager.showWhenStarting", False)  # Nascondi la finestra di download
profile.set_preference("browser.download.dir", dataset_folder)  # Cartella di download personalizzata
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/octet-stream")  # Tipi di file per il download automatico

# Aggiungi il profilo a Firefox
options.profile = profile

# Intestazioni per simulare una richiesta da un browser reale (Firefox)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0"
}

driver = None

def start_or_reset_session():
    global driver
    # Se la sessione è già avviata, prova a chiuderla
    if driver is not None:
        try:
            driver.quit()  # Prova a chiudere la sessione esistente
            print("Sessione esistente chiusa.")
        except WebDriverException:
            print("Nessuna sessione aperta da chiudere.")
    
    # Inizia una nuova sessione
    driver = webdriver.Firefox(service=service, options=options)
    print("Nuova sessione avviata.")

### Funzioni per estrarre le informazioni dato il link di Normattiva

In [41]:
# Namespace per lavorare sul file XML
namespaces = {
    'akn': 'http://docs.oasis-open.org/legaldocml/ns/akn/3.0',
    'eli': 'http://data.europa.eu/eli/ontology#',
    'fo': 'urn:oasis:names:tc:opendocument:xmlns:xsl-fo-compatible:1.0',
    'gu': 'http://www.gazzettaufficiale.it/eli/',
    'html': 'http://www.w3.org/1999/xhtml',
    'na': 'http://www.normattiva.it/eli/',
    'nakn': 'http://normattiva.it/akn/vocabulary',
    'nrdfa': 'http://www.normattiva.it/rdfa/',
    'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
    'rdfa': 'http://www.w3.org/1999/xhtml#'
}

# Funzione per correggere il link se manca il protocollo (http o https)
def correct_link_scheme(url, base_url):
    if not url.startswith("http://") and not url.startswith("https://"):
        url = urljoin(base_url, url)
    return url

def extract_details_from_urn(href):
    # Estrai l'URN dal link
    urn_start = href.find("urn:nir:")
    if urn_start == -1:
        print("URN non trovato nel link.")
        return None, None, None

    # Prendi tutto ciò che c'è dopo "urn:nir:"
    urn = href[urn_start:]  
    print(f"URN estratto: {urn}")
    
    parts = urn.split(':')

    if len(parts) >= 5:
        tipo_documento = parts[3].replace('.', ' ').upper()  # Decreto o altro

        try:
            data_documento, numero_documento = parts[4].split(';')  # data e numero separati da ;
        except ValueError:
            print("Errore nel formato dei dettagli (data e numero).")
            return None, None, None

        try:
            # Controlla il formato della data e parsala
            if '-' not in data_documento:  # Solo anno
                date_obj = datetime.datetime.strptime(data_documento, '%Y')
                formatted_date = date_obj.strftime('%Y')
            elif data_documento.count('-') == 1:  # Anno e mese
                date_obj = datetime.datetime.strptime(data_documento, '%Y-%m')
                formatted_date = date_obj.strftime('%B %Y')  # Es. "November 2024"
            elif data_documento.count('-') == 2:  # Anno, mese e giorno
                date_obj = datetime.datetime.strptime(data_documento, '%Y-%m-%d')
                formatted_date = date_obj.strftime('%d %B %Y')  # Es. "19 November 2024"
            else:
                print("Formato data non riconosciuto.")
                return None, None, None
        except Exception as e:
            print(f"Errore nella formattazione della data: {e}")
            return None, None, None

        return tipo_documento, formatted_date, numero_documento

    print("Formato dell'URN non valido.")
    return None, None, None
    
# Funzione per trovare il file più recente nella cartella Dataset
def find_most_recent_file(folder_path):
    files = os.listdir(folder_path)
    if not files:
        return None
    try:
        # Trova il file con la data di modifica più recente
        most_recent_file = max(
            (os.path.join(folder_path, f) for f in files),
            key=os.path.getmtime
        )
    except Exception as e:
        print(f"Errore durante la ricerca del file più recente: {e}")
        return None
    return most_recent_file

# Funzione per aspettare il file xml e nel caso returnare il file se trovato
def wait_for_recent_xml(dataset_folder):
    recent_file = None
    dot_count = 0  # Contatore dei puntini
    limit = 30 # Limite di tentativi
    counter = 0
    
    while not recent_file or not recent_file.endswith('.xml'):
        # Stampa la riga con i puntini e aggiorna la stessa riga
        print(f"\rAttesa del completamento del download per il file XML{'.' * dot_count}", end="")
        
        # Incrementa il numero di puntini fino a 3, poi resetta
        dot_count = (dot_count + 1) % 4
        time.sleep(1)
        
        # Verifica il file più recente
        recent_file = find_most_recent_file(dataset_folder)
        counter += 1
        if counter >= limit:
            print("\nNessun file XML trovato dopo diversi tentativi. Tentiamo con un altro metodo.")
            return None

    # Stampa il messaggio finale una volta trovato il file
    print(f"\nFile XML trovato: {recent_file}")
    return recent_file

### Funzioni per salvare il contenuto del link di Normattiva:

In [42]:
def extract_text_from_p(p):
    text = ""
    
    # Aggiungi il testo iniziale dell'elemento <p> se esiste
    if p.text:
        text += p.text.replace('\n', ' ') + " "
    
    # Itera sugli elementi figli di <p> per gestire <ins> e <ref>
    for element in p.iter():
        if element.tag == '{http://docs.oasis-open.org/legaldocml/ns/akn/3.0}ins':
            # Aggiungi il testo di <ins> e tail (parte di testo dopo <ins>) rimuovendo gli a capo
            text += (element.text or "").replace('\n', ' ') + " "
            if element.tail:
                text += element.tail.replace('\n', ' ') + " "
        elif element.tag == '{http://docs.oasis-open.org/legaldocml/ns/akn/3.0}ref':
            # Aggiungi il testo di <ref> e tail rimuovendo gli a capo
            text += (element.text or "").replace('\n', ' ') + " "
            if element.tail:
                text += element.tail.replace('\n', ' ') + " "
    
    # Aggiungi il testo tail di <p> se esiste
    if p.tail:
        text += p.tail.replace('\n', ' ') + " "
    
    return text.strip()  # Rimuove eventuali spazi iniziali o finali

# Funzione per formattare il numero articolo, rimpiazzando spazi con trattini.
def format_article_number(text):
    return text.lower().replace(" ", "-")

# Funzione per salvare il contenuto di un paragrafo in un file
def save_main_text_to_file(main_text, article_folder):
    file_name = f'main_text.txt'
    with open(os.path.join(article_folder, file_name), 'w', encoding='utf-8') as file:
        file.write(main_text.strip() + '\n')
    print(f"Salvato {file_name} in {article_folder}")

# Funzione per analizzare e salvare su file .txt la normativa
def analyze_file(file_path,folder_path):
    # Verifica che il file sia accessibile e attendi se necessario
    retries = 3
    while retries > 0:
        if os.access(file_path, os.R_OK):  # Verifica se il file è leggibile
            break
        else:
            print(f"File non accessibile, tentativo in corso. Tentativi rimanenti: {retries}")
            time.sleep(2)  # Attendi un po' prima di riprovare
            retries -= 1
    if retries == 0:
        print("Impossibile accedere al file dopo diversi tentativi.")
        return False
    # Carica e analizza il file XML
    try:
        tree = etree.parse(file_path)
    except OSError as e:
        print(f"Errore durante la lettura del file '{file_path}': {e}")
        return False
    except etree.XMLSyntaxError as e:
        print(f"Errore XML: il file '{file_path}' potrebbe essere vuoto o malformato: {e}")
        return False
    root = tree.getroot()
    # Trova tutti gli elementi <article> all'interno del file XML
    articles = root.findall('.//akn:article', namespaces)
    # print(f"Numero di articoli trovati: {len(articles)}")
    # Estrai e scrivi il testo di ciascun comma (paragrafo) all'interno degli articoli
    for article_index, article in enumerate(articles, start=1):
        # Ottieni l'eId dell'articolo
        article_eid = article.get('eId')
        # print(f"{article_eid}")
        # Crea una cartella per l'articolo
        article_folder = os.path.join(folder_path, article_eid)
        os.makedirs(article_folder, exist_ok=True)
        # Trova tutti gli elementi <paragraph> all'interno dell'articolo
        paragraphs = article.findall('.//akn:paragraph', namespaces)
        paragraph_counter = 1  # Inizializza il contatore dei paragrafi
        for paragraph in paragraphs:
            # Controlla se il paragrafo ha un elemento <num>
            # num = paragraph.find('.//akn:num', namespaces)
            # if num is None:
            #     continue  # Salta il paragrafo se non ha un elemento <num>
            # Inizializza il testo del paragrafo
            paragraph_text = ""
            # Verifica se è una lista o no
            lists = paragraph.findall('.//akn:list', namespaces)
            # Se è una lista, salva il contenuto in maniera diversa
            if lists:
                for lst in lists:
                    list_text = ""
                    intro = lst.find('.//akn:intro', namespaces)
                    if intro is not None:
                        ps = intro.findall('.//akn:p', namespaces)
                        for p in ps:
                            list_text += extract_text_from_p(p)
                    points = lst.findall('.//akn:point', namespaces)
                    for point in points:
                        point_num = point.find('.//akn:num', namespaces)
                        point_content = point.find('.//akn:content', namespaces)
                        if point_num is not None:
                            list_text += (point_num.text or "") + " "
                        if point_content is not None:
                            ps = point_content.findall('.//akn:p', namespaces)
                            for p in ps:
                                list_text += extract_text_from_p(p)
                    paragraph_text += list_text
            else:
                # Trova tutti gli elementi <p> all'interno del <content> del paragrafo
                content = paragraph.find('.//akn:content', namespaces)
                if content is not None:
                    ps = content.findall('.//akn:p', namespaces)
                    for p in ps:
                        paragraph_text += extract_text_from_p(p)
            # Se il paragrafo non è vuoto, scrivi il testo completo nel file
            if paragraph_text.strip():
                file_name = f'comma_{paragraph_counter}.txt'
                with open(os.path.join(article_folder, file_name), 'w', encoding='utf-8') as file:
                    file.write(paragraph_text.strip() + '\n')
                paragraph_counter += 1  # Incrementa il contatore solo se il file è stato creato
        # Trova tutti gli elementi <authorialNote> all'interno dell'articolo
        authorial_notes = article.findall('.//akn:authorialNote', namespaces)
        for note_index, note in enumerate(authorial_notes, start=1):
            note_text = ""
            note_ps = note.findall('.//akn:p', namespaces)
            for p in note_ps:
                note_text += extract_text_from_p(p)
            # Scrivi la nota in un file separato
            note_file_name = f'nota_articolo_{note_index}.txt'
            with open(os.path.join(article_folder, note_file_name), 'w', encoding='utf-8') as note_file:
                note_file.write(note_text.strip() + '\n')
    return True

### Funzioni per ricerca e download di riferimenti dentro i vari articoli (ricorsivo)

In [43]:


# Funzione per salvare il contenuto di un articolo in un file .txt
def save_text_to_file(text, folder, content_reference):
    content_reference = content_reference.replace("/", "-")
    file_path = os.path.join(folder, f"{content_reference}.txt")
    
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text)
    print(f"Contenuto salvato in: {file_path}")

# Funzione per eseguire la ricerca ricorsiva dei link all'interno degli articoli
def recursive_find(href, folder, depth, max_depth, content_reference):
    if depth > max_depth:
        return  # Se la profondità supera il massimo, fermati

    # Fai la richiesta HTTP per ottenere l'HTML della pagina
    response = requests.get(href)
    if response.status_code != 200:
        print(f"Errore nel caricamento della pagina: {href}")
        start_or_reset_session()
        response = requests.get(href)
        if response.status_code != 200:
            print(f"Errore nel caricamento della pagina. di nuovo: {href}")
            return

    # Usa BeautifulSoup per analizzare l'HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Estrai il div 'bodyTesto' principale
    body_testo_div = soup.find('div', class_="bodyTesto")
    if body_testo_div:
        print(f"{folder}")
        article_name = folder.split("/")[-1]  # Estrai il nome dell'articolo
        print(f"\nEstrazione contenuti per l'articolo: {article_name}")

        # Salva il testo dell'articolo
        save_text_to_file(content_reference + "\n" + body_testo_div.get_text(strip=True), folder, content_reference)
        
        # Trova tutti i link nel bodyTesto che contengono "~art"
        href_links = [
            {'href': a['href'], 'content': a.get_text(strip=True)} 
            for a in body_testo_div.find_all('a', href=True) 
            if "~art" in a['href']
        ]
        print(f"Trovati {len(href_links)} link nell'articolo {article_name}")

        # Per ogni link, chiamerai ricorsivamente la funzione
        for link in href_links:   
            full_link = urljoin("https://www.normattiva.it", link['href'])  # Costruisci il link completo
            recursive_find(full_link, folder, depth + 1, max_depth, link['content'])

# Funzione per trovare i riferimenti all'interno della pagina principale
def find_reference(href, main_folder, max_depth):
    article_links = []  # Lista per salvare i link degli articoli
    # Carica la pagina principale (href) con Selenium per la navigazione iniziale
    driver.get(href)
    time.sleep(2)  # Attendi il caricamento della pagina

    # Ottieni il contenuto della pagina
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Trova il div con id "albero"
    albero_div = soup.find('div', id="albero")
    if not albero_div:
        print("Div con id 'albero' non trovato.")
        return

    # Trova il primo ul all'interno del div
    ul = albero_div.find('ul')
    if not ul:
        print("Nessun <ul> trovato all'interno del div 'albero'.")
        return

    # Lista per salvare i dati estratti
    lista_articoli = []

    # Trova ogni <li> all'interno dell'<ul> e controlla se contiene un <a class="numero_articolo">
    for li in ul.find_all('li'):
        a_tag = li.find('a', class_="numero_articolo")
        if a_tag and 'onclick' in a_tag.attrs:
            # Estrai il link dall'attributo onclick
            onclick_content = a_tag['onclick']
            match = re.search(r"return showArticle\('([^']+)", onclick_content)
            if match:
                link_articolo = urljoin("https://www.normattiva.it", match.group(1))

                # Format articolo e aggiungi alla lista
                articolo_formattato = a_tag.text.lower().replace(" ", "-")
                lista_articoli.append({'link': link_articolo, 'articolo': articolo_formattato})

    # Apre una finestra secondaria all'inizio
    driver.execute_script("window.open('');")
    window_handles = driver.window_handles
    main_window = window_handles[0]  # Salva l'handle della finestra principale
    secondary_window = window_handles[-1]  # Salva l'handle della finestra secondaria

    # Per ogni articolo nella lista, usa la finestra secondaria per raccogliere i link e avviare la ricerca
    for articolo in lista_articoli:

        # Passa alla finestra secondaria per caricare l'articolo
        driver.switch_to.window(secondary_window)

        # Usa la finestra secondaria per caricare la pagina dell'articolo
        driver.get(articolo['link'])
        time.sleep(2)  # Attendi il caricamento della pagina

        # Ottieni il contenuto dell'articolo
        soup_articolo = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Estrai i link dal bodyTesto dell'articolo e inizia la ricerca ricorsiva
        body_testo_div = soup_articolo.find('div', class_="bodyTesto")
        if body_testo_div:
            print(f"\nInizio ricerca ricorsiva per l'articolo: {articolo['articolo']}")
        # Trova tutti i link dentro il bodyTesto e avvia la funzione ricorsiva per ciascun link
        href_links = [
            {'href': a['href'], 'content': a.get_text(strip=True)} 
            for a in body_testo_div.find_all('a', href=True) 
            if "~art" in a['href']
        ]
        article_links.append({'article': f"art_{articolo['articolo']}",
                                  'links': href_links})    
    
    driver.close() # Chiudi la finestra secondaria e torna alla finestra principale

    start_or_reset_session()

    article_links.sort(key=lambda x: x['article'])

    print("Inizio ricerca ricorsiva per i riferimenti trovati.")
    print(f"Numero di articoli trovati: {len(article_links)}")

    for elem in article_links:
        article_folder = os.path.join(main_folder, elem['article'])
        for link in elem['links']:
            full_link = urljoin("https://www.normattiva.it", link['href'])
            recursive_find(full_link, article_folder, 0, max_depth, link['content'])
    
    start_or_reset_session()
    

### Funzioni per scaricare il contenuto del link di Normattiva quando non è presente il file akn

In [44]:
# Funzione per salvare il contenuto di un paragrafo in un file
def save_main_text_to_file(main_text, article_folder):
    file_name = f'main_text.txt'
    with open(os.path.join(article_folder, file_name), 'w', encoding='utf-8') as file:
        file.write(main_text.strip() + '\n')
    print(f"Salvato {file_name} in {article_folder}")

# Funzione per cercare gli articoli, aprire ogni link, estrarre e salvare il contenuto
def download_article_without_akn_file(href, main_folder):
    start_or_reset_session()
    base_url = "https://www.normattiva.it"
    article_links = {}

    # Carica la pagina principale e attende che si carichi completamente
    print(f"Caricamento della pagina principale: {href}")
    for attempt in range(4):
        try:
            print(f"Tentativo {attempt + 1} di caricare la pagina: {href}")
            driver.get(href)
            time.sleep(1)  # Puoi rimuoverlo o ridurlo se hai attese più specifiche con WebDriverWait
            return True  # Caricamento riuscito
        except (TimeoutException, WebDriverException) as e:
            print(f"Errore durante il caricamento: {e}. Ritento tra 5 secondi...")
            time.sleep(5)
    if attempt == 3:
        print("Impossibile caricare la pagina dopo vari tentativi.")
        start_or_reset_session()
        download_article_without_akn_file(href, main_folder)
        return


    # Analizza la pagina con BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    albero_div = soup.find("div", id="albero")
    if not albero_div:
        print("Errore: div con id 'albero' non trovato.")
        return article_links

    # Trova e filtra gli <li> che contengono un <a> con classe 'numero_articolo'
    list_items = albero_div.find_all("li")
    filtered_items = [
        li for li in list_items 
        if li.find("a", class_="numero_articolo") and 
        not any(x in li.find("a", class_="numero_articolo").text for x in ["orig.", "agg."])
    ]
    
    print(f"Numero di elementi trovati con 'numero_articolo': {len(filtered_items)}")

    # Ottieni l'handle della finestra principale
    main_window = driver.current_window_handle

    for i, li in enumerate(filtered_items):
        article_element = li.find("a", class_="numero_articolo")

        if not article_element or 'onclick' not in article_element.attrs:
            print(f"Elemento {i} saltato: manca un link valido o l'attributo 'onclick'.")
            continue

        # Estrai il testo del numero articolo e formattalo
        article_number_text = article_element.text.strip()
        formatted_article_number = format_article_number(article_number_text)

        # Estrai il parametro dell'URL dall'attributo onclick
        onclick_text = article_element['onclick']
        match = re.search(r"showArticle\('([^']*)'", onclick_text)
        if not match:
            print(f"Elemento {i}: URL non trovato nell'onclick.")
            continue
        
        relative_url = match.group(1)
        article_url = base_url + relative_url
        print(f"Elemento {i} - URL dell'articolo: {article_url}")

        # Chiudi la scheda secondaria se già esistente
        if len(driver.window_handles) > 1:
            driver.switch_to.window(driver.window_handles[1])
            driver.close()

        # Apri una nuova scheda e passa a essa
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])  # Passa alla scheda secondaria
        driver.get(article_url)

        # Attendi che il div bodyTesto sia presente
        try:
            body_text_div = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CLASS_NAME, "bodyTesto"))
            )
            article_soup = BeautifulSoup(driver.page_source, 'html.parser')
            body_text_div = article_soup.find("div", class_="bodyTesto")

            article_folder = os.path.join(main_folder, f"art_{formatted_article_number}")
            os.makedirs(article_folder, exist_ok=True)

            # Estrai tutto il testo dal bodyTesto
            full_text = body_text_div.get_text(separator=" ").strip()

            save_main_text_to_file(full_text, article_folder)

        except TimeoutException:
            print(f"Elemento {i}: div con classe 'bodyTesto' non trovato dopo attesa.")
            driver.close()
            driver.switch_to.window(main_window)
            continue

        # Chiudi la scheda secondaria e torna alla finestra principale
        driver.close()
        driver.switch_to.window(main_window)

    print("Download e salvataggio completati per tutti gli articoli.")

### Funzioni per effettuare il download del contenuto del link:

In [45]:
def inizialize_folder(href):
    tipo_documento, formatted_date, numero_documento = extract_details_from_urn(href)
    if tipo_documento and formatted_date and numero_documento:
        # Formatta la cartella principale
        folder_name = f"{tipo_documento} {formatted_date},n. {numero_documento}"

        # Leggi il contenuto attuale del file per controllare duplicati
        with open(normattiva_links_file, 'r') as f:
            existing_links = f.readlines()
        
        # Verifica se il link è già presente
        new_entry = f"{href};{folder_name}\n"
        if new_entry not in existing_links:
            # Aggiungi il link e il nome della cartella al file NormattivaLinks
            with open(normattiva_links_file, 'a') as f:
                f.write(new_entry)
        
        print(f"Cartella da creare: {folder_name}")
        # Crea la cartella principale per il documento
        main_folder = os.path.join(os.path.expanduser('~'), 'Desktop', 'Dataset', folder_name)
        os.makedirs(main_folder, exist_ok=True)
        return main_folder, folder_name
    

def download_file_and_articles(href,main_folder,timeout=20, poll_interval=1):
    try:
        driver.get(href)
        start_time = time.time()
        download_link = None
        
        # Ciclo di polling per cercare l'elemento senza bloccare il codice
        while (time.time() - start_time) < timeout:
            download_buttons = driver.find_elements(By.XPATH, "//a[contains(@href, '/do/atto/caricaAKN?')]")
            if download_buttons:
                download_link = download_buttons[0]
                break
            # Attesa breve prima di riprovare
            time.sleep(poll_interval)
        
        if download_link:
            # Se il pulsante è stato trovato, procedi con il download
            file_url = download_link.get_attribute('href')
            print(f"Link per il download trovato: {file_url}")
            download_link.click()

            recent_file = wait_for_recent_xml(dataset_folder)
            if recent_file:
                response = analyze_file(recent_file, main_folder)
                if response is False:
                    print("Errore durante l'analisi del file XML. Provo con un altro metodo.")
                    download_article_without_akn_file(href, main_folder)
                # Elimina il file XML scaricato dopo averlo elaborato
                os.remove(recent_file)
                print(f"File XML eliminato: {recent_file}")
            else:
                download_article_without_akn_file(href, main_folder)
                
        else:
            # Se il pulsante non è stato trovato entro il tempo limite, esegui la funzione alternativa
            print("Non c'è il pulsante per il download. Provo con un altro metodo.")
            download_article_without_akn_file(href, main_folder)
            
        
    finally: 
        print("Download completato")

### Cerchiamo la norma e la scarichiamo:

In [46]:
# Definiamo quanto profondo deve essere lo scraping dei link

def main_function_download(link_name,link_finded):
    
    # Leggi il contenuto attuale del file per controllare duplicati
    with open(normattiva_links_file, 'r') as f:
        existing_links = f.readlines()
    
    # Verifica se il link è già presente
    new_entry = f"{link_finded};{link_name}\n"
    if new_entry not in existing_links:
        # Aggiungi il link e il nome della cartella al file NormattivaLinks
        with open(normattiva_links_file, 'a') as f:
            f.write(new_entry)
    
    print(f"Cartella da creare: {link_name}")
    # Crea la cartella principale per il documento
    main_folder = os.path.join(os.path.expanduser('~'), 'Desktop', 'Dataset', link_name)
    os.makedirs(main_folder, exist_ok=True)
    
    download_file_and_articles("https://www.normattiva.it"+link_finded, main_folder)

In [47]:
def estrai_href(html):
    soup = BeautifulSoup(html, "html.parser")
    # Trova tutti i tag <a> con il titolo specifico
    link_atti = soup.find_all("a", title="Dettaglio atto")
    # Rimuove newline, tab e spazi multipli dal testo
    def pulisci_testo(testo):
        return " ".join(testo.split())  # Divide in parole e riunisce rimuovendo extra spazi e newline
    
    return [(a["href"], pulisci_testo(a.get_text(strip=True))) for a in link_atti]

# Funzione per verificare se la pagina contiene un errore
def contiene_errore(html):
    soup = BeautifulSoup(html, "html.parser")
    # Verifica se c'è un div con class="alert alert-danger"
    errore = soup.find("div", class_="alert alert-danger")
    return errore is not None

# Funzione per inizializzare la ricerca e ottenere l'URL di base
def setup_search():
    # Passo 1: Naviga alla homepage
    driver.get("https://www.normattiva.it/home")

    # Trova il campo di ricerca usando l'XPath e inserisci il testo
    campo_ricerca = driver.find_element(By.XPATH, '//*[@id="testoRicerca"]')

    search_query = input("Inserisci il testo da cercare: ")

    campo_ricerca.send_keys(search_query)

    # Trova il pulsante usando l'XPath e cliccalo
    pulsante_ricerca = driver.find_element(By.XPATH, '//*[@id="button-3"]')
    pulsante_ricerca.click()

    # Aspetta che la pagina si carichi
    time.sleep(5)

    # Ottieni l'URL dinamico della pagina di risultati
    url_base = driver.current_url
    print(f"URL di base generato: {url_base}")
    return url_base

def download_link_references():
    start_or_reset_session()

    try:

        url_base = setup_search()    

        # Passo 2: Inizia il ciclo per le pagine successive
        page = 0
        articoli_totali = []

        while True:
            # Genera l'URL per la pagina corrente
            url_pagina = url_base.replace("0?", f"{page}?")
            print(f"Caricamento pagina: {url_pagina}")
            
            # Apri la pagina nel browser
            driver.get(url_pagina)
            time.sleep(3)  # Aspetta il caricamento della pagina

            # Ottieni l'HTML della pagina
            html = driver.page_source
            
            # Estrai gli href
            href_trovati = estrai_href(html)

            if contiene_errore(html):
                print("Errore rilevato nella pagina, riavviamo il ciclo.")
                time.sleep(3)
                url_base = setup_search()
            else:
                # Se non ci sono href, si presume che non ci siano più pagine
                if not href_trovati:
                    print("Nessun altro articolo trovato, fine del ciclo.")
                    break

                for href,testo in href_trovati:
                    print(f"Link: {href}, Testo: {testo}")
                # Aggiungi gli href trovati al totale
                articoli_totali.extend(href_trovati)

                # Stampa gli href della pagina corrente
                print(f"Pagina {page}: Trovati {len(href_trovati)} articoli.")

                # Passa alla pagina successiva
                page += 1

        # Stampa il numero totale di articoli
        print(f"Totale articoli trovati: {len(articoli_totali)}")
        print("Elenco articoli:")
        desktop_path = os.path.expanduser("~/Desktop")
        file_path = os.path.join(desktop_path, "linksTrovati.txt")
        for articolo,testo in articoli_totali:
            print(f"{href},{testo}")
        
        with open(file_path, "w", encoding="utf-8") as f:
            for href, testo in articoli_totali:
                f.write(f"{href}: {testo}\n")
        
    finally:
        # Chiudi il driver
        driver.quit()

In [48]:
MAX_DEPTH = 1

def carica_links_da_file(percorso_file):
    links = []  # Lista per memorizzare i dati
    with open(percorso_file, "r", encoding="utf-8") as file:
        for linea in file:
            # Rimuove newline e spazi inutili
            linea = linea.strip()
            if ": " in linea:  # Verifica che il formato sia corretto
                href, testo = linea.split(": ", 1)  # Divide la stringa in href e testo
                links.append({"link": href, "testo": testo})  # Aggiunge un dizionario alla lista
    return links

start_or_reset_session()
# download_link_references()
# # Percorso relativo al Desktop
# percorso_file = os.path.expanduser("~/Desktop/linksTrovati.txt")

# # Carica i link dal file
# lista_links = carica_links_da_file(percorso_file)
    
# print(f"numero di elementi: {len(lista_links)}")

# for elemento in lista_links:
#     # main_function_download(elemento["testo"],elemento["link"])
#     folder = os.path.join(dataset_references_folder, elemento["testo"])
#     os.makedirs(folder, exist_ok=True)
#     find_reference("https://www.normattiva.it/"+elemento["link"],folder,MAX_DEPTH)
#     time.sleep(3)

# main_function_download("DECRETO LEGISLATIVO 31 marzo 2023, n. 36","/uri-res/N2Ls?urn:nir:stato:decreto.legislativo:2023;036")
folder = os.path.join(dataset_folder, "DECRETO LEGISLATIVO 31 marzo 2023, n. 36")
find_reference("https://www.normattiva.it/uri-res/N2Ls?urn:nir:stato:decreto.legislativo:2023;036",folder,MAX_DEPTH)
driver.quit()

Nuova sessione avviata.

Inizio ricerca ricorsiva per l'articolo: 1

Inizio ricerca ricorsiva per l'articolo: 2

Inizio ricerca ricorsiva per l'articolo: 3

Inizio ricerca ricorsiva per l'articolo: 4

Inizio ricerca ricorsiva per l'articolo: 5

Inizio ricerca ricorsiva per l'articolo: 6

Inizio ricerca ricorsiva per l'articolo: 7

Inizio ricerca ricorsiva per l'articolo: 8

Inizio ricerca ricorsiva per l'articolo: 9

Inizio ricerca ricorsiva per l'articolo: 10

Inizio ricerca ricorsiva per l'articolo: 11

Inizio ricerca ricorsiva per l'articolo: 12

Inizio ricerca ricorsiva per l'articolo: 13

Inizio ricerca ricorsiva per l'articolo: 14

Inizio ricerca ricorsiva per l'articolo: 15

Inizio ricerca ricorsiva per l'articolo: 16

Inizio ricerca ricorsiva per l'articolo: orig.

Inizio ricerca ricorsiva per l'articolo: 17

Inizio ricerca ricorsiva per l'articolo: 18

Inizio ricerca ricorsiva per l'articolo: 19

Inizio ricerca ricorsiva per l'articolo: 20

Inizio ricerca ricorsiva per l'artic

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\burre\\Desktop\\Dataset\\DECRETO LEGISLATIVO 31 marzo 2023, n. 36\\art_agg.1\\articolo 46-bis del codice delle pari opportunità tra uomo e donna.txt'